# Test whether the simplification of the Transition Matrix works

In [1]:
import numpy as np
from scipy.linalg import expm

# Load the traditional Transition Matrix
Transition matrix t[i,j]: Infinitesimal rate of jumping from state i to state j

In [2]:
roh_in = 0.0005     # The rate of jumping to another Haplotype
roh_out = 0.001     # The rate of jumping out
roh_jump = 0.02   # The rate of jumping within ROH
n=20 # Nr of Reference Haplotypes

In [3]:
def calc_transitions(rate=True):
        """Return Transition Matrix to exponate.
        n: Nr of Reference Haplotypes"""

        t_mat = -np.ones((n + 1, n + 1))  # Initialize Transition Matrix

        t_mat[1:, 0] = roh_out  # The rate of jumping out roh
        t_mat[0, 1:] = roh_in / n  # Jumping into any ROH State
        t_mat[1:, 1:] = roh_jump / n  # Jumping between ROH State

        ### Do the Diagonal (do the usual model - for inf. substract 1)
        di = np.diag_indices(n + 1)
        d = 1 - rate  # Whether to use 1 one the diagonal
        #print(f"Diagonal Factor: {d}")
        t_mat[di] =  d - roh_out - roh_jump + roh_jump / (n)  # Don't forget the self jump
        t_mat[0, 0] = d  - roh_in   # The rate of staying in diploid

        ### Sanity Check if everything was filled correctly
        if rate == True:
            assert(np.all(np.sum(t_mat, axis=1) > -0.0001))
            assert(np.all(np.sum(t_mat, axis=1) <  0.0001))
        return t_mat

In [4]:
t = calc_transitions()
print(np.shape(t))

(21, 21)


In [5]:
e = expm(t*0.1)    # The exact, exponentiated Version
e[:3,:3]

array([[9.99950004e-01, 2.49981251e-06, 2.49981251e-06],
       [9.99925004e-05, 9.98002094e-01, 9.98902021e-05],
       [9.99925004e-05, 9.98902021e-05, 9.98002094e-01]])

In [6]:
np.eye(3) + 0.1 * t[:3,:3]   # The older, linearized Version

array([[9.9995e-01, 2.5000e-06, 2.5000e-06],
       [1.0000e-04, 9.9800e-01, 1.0000e-04],
       [1.0000e-04, 1.0000e-04, 9.9800e-01]])

# Do the Simplified Version:

In [54]:
t_simple = prep_3x3matrix(t)
t_simple

array([[-5.00e-04,  2.50e-05,  4.75e-04],
       [ 1.00e-03, -2.00e-02,  1.90e-02],
       [ 1.00e-03,  1.00e-03, -2.00e-03]])

In [12]:
np.sum(t_simple, axis=1)  ### Sanity Check. Should sum to 0!

array([ 0.00000000e+00,  0.00000000e+00, -8.67361738e-19])

# Compare the different ways of calculating the Matrix Exponential

In [13]:
e = expm(t*10)    # The exact, exponentiated Version of the full Matrix
e[:3,:3]

array([[9.95037313e-01, 2.48134340e-04, 2.48134340e-04],
       [9.92537360e-03, 8.19558765e-01, 8.97451902e-03],
       [9.92537360e-03, 8.97451902e-03, 8.19558765e-01]])

In [49]:
e_new = expm(t_simple * 100)
e_new[:2,2] = e_new[:2,2] / (n-1)  # Normalize to transition rate for non-collapsed state
e_new[:3, :3]

array([[0.95356933, 0.00232153, 0.00232153],
       [0.09286135, 0.16169054, 0.03923411],
       [0.09286135, 0.03923411, 0.86790454]])

## Only the 2,2 entry, which is "staying in a different ROH" State is obviously different. But this one is never used in my calculations anyways

# Try with Eigenvalue Decomposition:

In [66]:
def prep_3x3matrix(t):
    """Prepares the 3x3 Matrix"""
    n = np.shape(t)[0] - 1 # Infer the Number of reference States
    t_simple = -np.ones((3,3))  # Initiate to -1 (for later Sanity Check if everything is filled)
    t_simple[:2, :2] = t[:2, :2]
    t_simple[2, 2]   = np.sum(t[2, 2:]) # The probability of staying when in there
    t_simple[:2, 2]  = t[:2,2] * (n-1) # Jumping into 3rd state: Sum over all reference states
    t_simple[2,:2] = t[2,:2] # The jumping out probability is the same
    return t_simple


def exponentiate_r(rates, rec_v):
    """Calculates exponentiation of the rates matrix with rec_v
    rates: 2D Matrix.
    rec_v: Array of length l"""
    eva, evec = np.linalg.eig(rates) # Do the Eigenvalue Decomposition
    assert(np.max(eva)<0)   # Some Sanity Check that Matrix is valid rate Matrix
    evec_r = np.linalg.inv(evec)    # Do the Inversion
    
    d = np.exp(rec_v[:, None] * eva)     # Create vector of the exponentiated diagonals 
    # Use some Einstein Sum Convention Fun (C Speed)
    res = np.einsum('...ik, ...k, ...kj ->...ij', evec, d, evec_r)
    return res

# Do the testing for Transformation Matrix

In [71]:
r_vec = np.array([1, 10, 100])
t_simple = prep_3x3matrix(t)
r = exponentiate_r(t_simple, r_vec)
r

array([[[9.99500375e-01, 2.49812594e-05, 4.74643928e-04],
        [9.99250375e-04, 9.80208054e-01, 1.87926958e-02],
        [9.99250375e-04, 9.89089253e-04, 9.98011660e-01]],

       [[9.95037313e-01, 2.48134340e-04, 4.71455246e-03],
        [9.92537360e-03, 8.19558765e-01, 1.70515861e-01],
        [9.92537360e-03, 8.97451902e-03, 9.81100107e-01]],

       [[9.53569325e-01, 2.32153373e-03, 4.41091408e-02],
        [9.28613490e-02, 1.61690539e-01, 7.45448112e-01],
        [9.28613490e-02, 3.92341111e-02, 8.67904540e-01]]])

In [69]:
r2 = np.array([expm(t_simple*r) for r in r_vec])
r2

array([[[9.99500375e-01, 2.49812594e-05, 4.74643928e-04],
        [9.99250375e-04, 9.80208054e-01, 1.87926958e-02],
        [9.99250375e-04, 9.89089253e-04, 9.98011660e-01]],

       [[9.95037313e-01, 2.48134340e-04, 4.71455246e-03],
        [9.92537360e-03, 8.19558765e-01, 1.70515861e-01],
        [9.92537360e-03, 8.97451902e-03, 9.81100107e-01]],

       [[9.53569325e-01, 2.32153373e-03, 4.41091408e-02],
        [9.28613490e-02, 1.61690539e-01, 7.45448112e-01],
        [9.28613490e-02, 3.92341111e-02, 8.67904540e-01]]])

In [67]:
r_vec = np.arange(10000)
%timeit exponentiate_r(t_simple, r_vec)

7.71 ms ± 1.23 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [76]:
rr = exponentiate_r(t_simple, r_vec)